# Reinforcement learning

## TD control: Q-Learning (off-policy)
* Problems to consider: Gridworld_environment
* Q-Learning: $Q(S,A) := Q(S,A) + \alpha \Big [ R(S') + \gamma \max_a Q(S′, a) − Q(S, A) \Big ]$
* Remarks: 
    * A is chosen from S using policy derived from Q (e.g., Softmax and ucb-1)
    * In Q-Learning, the "best" action a is taken in $\max_a Q(S′, a)$ irrespective of the control policy

# Simulated environment: The Grid-World environment
Consider the grid world environment in which an agent begins at an initial location (0,0) and tries to get to the target location (3,3), one step at a time. In the 4x4 grid world, the agent randomly selects any action 
* Action Space: The agent can take discrete actions: move up, down, left, and right.
* Rewards:
    * The agent gets a reward of -1 as it moves from one state to another after taking an action. This negative  reward (i.e. penalty) enforces the agent to complete the journey and reach the terminal state in minimum number of steps.
    * If the agent wants to run out of the grid, it gets a penalty of -5. Example, the agent receives a penalty of -5, if it goes LEFTward from any of these states (0, 0), (0, 1), (0, 2), (0, 3).
    * The agent also gets a reward of 1, whenever it moves to the terminal state (3,3) from any of these two states (2, 3), (3, 2).

### References:

Python Code snippet from Professor Stefan Fauser's jupyter notebook

In [31]:
import numpy as np

### environment Simulation

In [32]:
def gridworld_sim_env(S, A, optional_args=None, verbose=False):
    nrow, ncol = 4, 4  # 4x4 grid world environment
    
    if optional_args is not None:
        penalty_outside_grid = optional_args.get("penalty_outside_grid", -5)
        penalty_movt = optional_args.get("penalty_movt", -1)
        goal_reward= optional_args.get("goal_reward", 1)
    else:
        penalty_outside_grid = -5
        penalty_movt = -1
        goal_reward= 1
    
    if verbose: 
        print("S =", S, "A =", A)
    
    curr_row= S // ncol
    curr_col = S % ncol
    col_mx = curr_col - 1
    row_mx = curr_row - 1
    col_mn = curr_col + 1
    row_mn = curr_row + 1
    
    if A == 0:  # Go left
        next_col = max(0, col_mx)
        next_row = curr_row
    elif A == 1:  # Go up
        next_row = max(0, row_mx)
        next_col = curr_col
    elif A == 2:  # Go right
        next_col = min(ncol - 1, col_mn)
        next_row = curr_row
    elif A == 3:  # Go down
        next_row = min(nrow - 1, row_mn)
        next_col = curr_col
    
    if (next_row, next_col) == (3, 3):  # The Terminal state
        R = goal_reward
        S_prime = None  # Terminal state
    elif next_row < 0 or next_col < 0 or next_row >= nrow or next_col >= ncol:
        R = penalty_outside_grid  # reward for moving outside the grid
        S_prime = S  # remain in same state
    else:
        R = penalty_movt
        A_dd = next_row * ncol
        S_prime = A_dd + next_col
    
    if verbose:
        print("new r =", next_row, "new c =", next_col, "S_prime =", S_prime, "R =", R)
    
    return R, S_prime

def gridworld_sim_env_print_Q(Q):    
    for r in range(4):
        print("-------------------------\n", end="")
        for A in range(4):
            print("|", end="")
            for c in range(4):
                S = r * 4 + c
                if (r, c) == (3, 3):
                    print("G|", end="")  # goal
                elif A == 0:
                    print("L: {:06.2f}|".format(np.round(Q[S, A], 2)), end="")
                elif A == 1:
                    print("U: {:06.2f}|".format(np.round(Q[S, A], 2)), end="")
                elif A == 2:
                    print("R: {:06.2f}|".format(np.round(Q[S, A], 2)), end="")
                elif A == 3:
                    print("D: {:06.2f}|".format(np.round(Q[S, A], 2)), end="")
            print("\n", end="")
    print("-------------------------\n", end="")


def gridworld_sim_env_policy(Q):    
    for r in range(4):
        print("-------------------------\n|", end="")
        for c in range(4):
            S = r * 4 + c
            if (r, c) == (3, 3):
                print("G|", end="")  # goal
            else:
                rd_q = np.round(Q[S,], 2)
                A = np.argmax(rd_q)
                if A == 0:
                    print("L|", end="")
                elif A == 1:
                    print("U|", end="")
                elif A == 2:
                    print("R|", end="")
                elif A == 3:
                    print("D|", end="")
        print("\n", end="")
    print("-------------------------\n", end="")

In [33]:
gridworld_sim_env(0, 2, verbose=True) # go to the right (2) from start (0)

S = 0 A = 2
new r = 0 new c = 1 S_prime = 1 R = -1


(-1, 1)

In [34]:
#ucb-1 Function
def ucb_1(Q, N, t):
    slog = np.log(t + 1)
    tlog = N + 1e-6
    exploration_bonus = np.sqrt( slog / tlog)
    Q_exp = Q + exploration_bonus
    return Q_exp

def ucb_1_take_action(Q, N, S, t):
    slog = np.log(t + 1)
    tlog =  N[S] + 1e-6
    exploration_bonus = np.sqrt(slog / tlog)
    action_values = Q[S, :] + exploration_bonus
    return np.argmax(action_values)

def update_Q_ucb_1(Q, N, S, A, R, S_prime, alpha, gamma, t):
    if S_prime is None:  # Q[S_prime, ...] means terminal state
        Q[S, A] = Q[S, A] + alpha * (R - Q[S, A])
    else:  # Q-learning; search best action by using UCB-1
        best_action = np.argmax(ucb_1(Q[S_prime, :], N[S_prime, :], t))
        Q[S, A] = Q[S, A] + alpha * (R + gamma * Q[S_prime, best_action] - Q[S, A])
    return Q



def gridworld_sim_env_loop_ucb_1(episodes, alpha, gamma, optional_args=None):
    maximum_states = 16  # since grid world is a 4x4 grid
    maximum_actions = 4  # Actions: 0 - Left, 1 - Down, 2 - Right, 3 - Up
    Q = np.zeros((maximum_states, maximum_actions))
    N = np.zeros((maximum_states, maximum_actions))
    for episode in range(1, episodes + 1):
        S = 0  # start state = 0
        t = 1  # time step
        while S is not None:
            Q, N, S = agent_sim_env_interaction_ucb_1(gridworld_sim_env, optional_args, Q, N, S, None, t, alpha=alpha, gamma=gamma)
            t += 1
    return Q

def agent_sim_env_interaction_ucb_1(env_func, optional_args, Q, N, S, A, t, alpha=0.5, gamma=0.9):
    if A is None:
        A = ucb_1_take_action(Q, N, S, t)
    R, S_prime = env_func(S, A, optional_args=optional_args)
    Q = update_Q_ucb_1(Q, N, S, A, R, S_prime, alpha, gamma, t)
    N[S, A] += 1  # Update the action count
    return Q, N, S_prime



# Print the Q values for UCB-1 function in a grid world environment
print("Q-Learning with UCB-1 Exploration")
alpha = 0.5
gamma = 0.9
for T in [1000, 10000, 20000]:
    np.random.seed(777)
    Q_ucb_1 = gridworld_sim_env_loop_ucb_1(T, alpha, gamma, optional_args=None)  # Update optional_args
    print("Policy after", T, "episodes:")
    gridworld_sim_env_policy(Q_ucb_1)
    print("Q values:")
    gridworld_sim_env_print_Q(Q_ucb_1)


Q-Learning with UCB-1 Exploration
Policy after 1000 episodes:
-------------------------
|R|R|R|D|
-------------------------
|R|R|D|D|
-------------------------
|R|R|R|D|
-------------------------
|R|R|R|G|
-------------------------
Q values:
-------------------------
|L: -04.01|L: -03.26|L: -02.88|L: -02.13|
|U: -04.01|U: -03.37|U: -02.65|U: -01.85|
|R: -03.50|R: -02.78|R: -01.98|R: -01.85|
|D: -03.60|D: -02.90|D: -01.98|D: -01.09|
-------------------------
|L: -03.37|L: -03.07|L: -02.18|L: -01.48|
|U: -03.50|U: -02.83|U: -02.26|U: -01.30|
|R: -02.98|R: -02.12|R: -01.58|R: -01.43|
|D: -02.98|D: -02.28|D: -01.09|D: -00.10|
-------------------------
|L: -03.02|L: -02.40|L: -01.79|L: -00.80|
|U: -02.93|U: -02.29|U: -01.78|U: -00.72|
|R: -02.34|R: -01.30|R: -00.10|R: -00.50|
|D: -02.34|D: -01.88|D: -00.78|D: 001.00|
-------------------------
|L: -02.26|L: -01.77|L: -01.19|G|
|U: -02.26|U: -02.05|U: -00.72|G|
|R: -01.38|R: -00.18|R: 001.00|G|
|D: -02.26|D: -01.85|D: -00.98|G|
--------------

In [35]:
#Softmax Function
def softmax(x, temp=1.0):
    e_x = np.exp((x - np.max(x)) / temp)
    return e_x / e_x.sum()

def update_Q_softmax(Q, S, A, R, S_prime, alpha, gamma, temp):
    if S_prime is None:  # Q[S_prime, ...] means terminal state
        Q[S, A] = Q[S, A] + alpha * (R - Q[S, A])
    else:  #  # Q-learning; search best action by using softmax
        prob = softmax(Q[S_prime, :], temp)
        best_action = np.random.choice(len(prob), p=prob)
        Q[S, A] = Q[S, A] + alpha * (R + gamma * Q[S_prime, best_action] - Q[S, A])
    return Q


def softmax_take_action(Q, S, temp=1.0):
    prob = softmax(Q[S, :], temp)
    action = np.random.choice(len(prob), p=prob)
    return action

def agent_sim_env_interaction_softmax(env_func, optional_args, Q, S, A, temp=1.0, alpha=0.5, gamma=0.9):
    if A is None:
        A = softmax_take_action(Q, S, temp)
    R, S_prime = env_func(S, A, optional_args=optional_args)
    Q = update_Q_softmax(Q, S, A, R, S_prime, alpha, gamma, temp)
    return Q, S_prime

def gridworld_sim_env_loop_softmax(episodes, alpha, gamma, temp=1.0, optional_args=None):
    maximum_states = 16  # since grid world is a 4x4 grid
    maximum_actions = 4  # Actions: 0 - Left, 1 - Down, 2 - Right, 3 - Up
    Q = np.zeros((maximum_states, maximum_actions))

    for _ in range(episodes):
        S = 0  # start state = 0
        A = None
        while S is not None:
            Q, S = agent_sim_env_interaction_softmax(gridworld_sim_env, optional_args, Q, S, A, temp=temp, alpha=alpha, gamma=gamma)
    return Q


# Print the Q values for softmax function in the grid world environment
print("Q-Learning with Softmax Exploration")
episodes_list = [1000, 10000, 20000]
alpha = 0.1
gamma = 0.5

for T in episodes_list:
    np.random.seed(777)
    Q = gridworld_sim_env_loop_softmax(T, alpha, gamma, temp=1.0, optional_args=None)
    print(f"Policy after {T} episodes:")
    gridworld_sim_env_policy(Q)
    print("Q values:")
    gridworld_sim_env_print_Q(Q)

Q-Learning with Softmax Exploration
Policy after 1000 episodes:
-------------------------
|R|D|D|D|
-------------------------
|R|D|D|D|
-------------------------
|R|R|D|D|
-------------------------
|R|R|R|G|
-------------------------
Q values:
-------------------------
|L: -02.00|L: -02.00|L: -01.99|L: -01.96|
|U: -02.00|U: -01.99|U: -01.97|U: -01.91|
|R: -01.99|R: -01.97|R: -01.90|R: -01.93|
|D: -01.99|D: -01.95|D: -01.86|D: -01.66|
-------------------------
|L: -01.99|L: -01.99|L: -01.97|L: -01.88|
|U: -02.00|U: -01.99|U: -01.96|U: -01.90|
|R: -01.96|R: -01.87|R: -01.63|R: -01.62|
|D: -01.98|D: -01.86|D: -01.55|D: -00.69|
-------------------------
|L: -01.97|L: -01.97|L: -01.87|L: -01.50|
|U: -01.99|U: -01.96|U: -01.89|U: -01.61|
|R: -01.87|R: -01.49|R: -00.90|R: -00.75|
|D: -01.91|D: -01.63|D: -00.67|D: 001.00|
-------------------------
|L: -01.94|L: -01.93|L: -01.58|G|
|U: -01.96|U: -01.88|U: -01.52|G|
|R: -01.67|R: -00.75|R: 001.00|G|
|D: -01.93|D: -01.77|D: -00.77|G|
------------

In [36]:
# left hand and right hand values should be equal
print(Q[0, 2], -1 + gamma * Q[0, 3]) # go right from upper left state
print(Q[0, 1], -1 + gamma * Q[0, 1]) # go up from start state

-1.9925590113476528 -1.9963418282157885
-1.9969489923084016 -1.9984744961542007


### Hyperparameter Tuning Using Bayesian Optimization

In [37]:
# function for Gaussian Process
def kernel(X1, X2, l=1e-2, sigma_f=1.0):
    distance_matrix = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    sig_w = np.exp(-0.5 / l**2 * distance_matrix)
    return sigma_f**2 * sig_w # Calculate the Gaussian kernel matrix using RBF

# fitting a Gaussian Process model to the provided data
def fit_guassian_process(X, y, l=1e-2, sigma_f=1.0):
    K_nel = kernel(X, X, l)
    m_eye = 1e-9 * np.eye(len(X))
    K = K_nel + m_eye
    L = np.linalg.cholesky(K)
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, y))
    
    # use new input data (X_in) to make predictions
    def predict(X_in): 
       k_in = kernel(X, X_in, l)
       Lk = np.linalg.solve(L,k_in)
       predicted_mean = np.dot(Lk.T, alpha)

       k_in2 = kernel(X_in, X_in, l)
       cov = k_in2 - np.dot(Lk.T, Lk)
       return predicted_mean, cov

    return predict
#Objective function
def objective_func(acquisition_func, *hyperparameters, **kwargs):
    if acquisition_func == 'ucb-1':
        alpha, gamma, *rest = hyperparameters
        neg_ht_ucb_1 = ((alpha - 0.5)**2 + (gamma - 0.5)**2) * -1
        return neg_ht_ucb_1  # Objective for UCB-1
    elif acquisition_func == 'softmax':
        alpha, gamma, temp = hyperparameters
        neg_ht_softmax = (((alpha - 0.5)**2 + (gamma - 0.5)**2) + (temp - 1.0)**2) * -1
        return neg_ht_softmax  # Objective for Softmax
    else:
        raise ValueError("Invalid acquisition function. Select 'ucb-1' or 'softmax'.")


def softmax_acquisition_func(mean, std, temp=1.0):
    mx_mn = mean - np.max(mean)
    std_tmp = temp * std + 1e-9
    exponentiated_values = np.exp(mx_mn / std_tmp)
    prob = exponentiated_values / np.sum(exponentiated_values)
    return prob

def bayesian_optimization(n_iterations, bound, acquisition_func='ucb-1', **kwargs):
    np.random.seed(777)
    X = np.random.rand(n_iterations, len(bound))
    y = np.zeros(n_iterations)

    for i in range(n_iterations):
        x_in_hyperparameter_space = bound[:, 0] + X[i] * (bound[:, 1] - bound[:, 0])

        y[i] = objective_func(acquisition_func, *x_in_hyperparameter_space, **kwargs)

        gp_predict = fit_guassian_process(X[:i + 1], y[:i + 1])
        X_in = np.random.rand(1, len(bound))  # Generate a random point for prediction
        mean, cov = gp_predict(X_in)

        if acquisition_func == 'ucb-1':
            acquisition_values = ucb_1_acquisition_func(mean, np.sqrt(np.diag(cov)))
        elif acquisition_func == 'softmax':
            acquisition_values = softmax_acquisition_func(mean, np.sqrt(np.diag(cov)), **kwargs)
        else:
            raise ValueError("Invalid acquisition function. Choose 'ucb-1' or 'softmax'.")

        # Select next point with maximum acquisition value
        next_point = X[np.argmax(acquisition_values)]

        x_in_hyperparameter_space = bound[:, 0] + next_point * (bound[:, 1] - bound[:, 0])
        y_next = objective_func(acquisition_func, *x_in_hyperparameter_space, **kwargs)

        X[i] = next_point
        y[i] = y_next

    best_index = np.argmax(y)
    best_hyperparameters = bound[:, 0] + X[best_index] * (bound[:, 1] - bound[:, 0])

    return best_hyperparameters

def ucb_1_acquisition_func(mean, std, exploration_tradeoff=0.1):
    mn_expo = mean + exploration_tradeoff * std
    return mn_expo

# bound for hyperparameters
bound = np.array([[0.1, 0.9], [0.3, 0.9], [0.1, 2.0]])

# Bayesian Optimization for UCB-1 using Gaussian Process
best_hyperparameters_ucb_1 = bayesian_optimization(n_iterations=100, bound=bound[:3], acquisition_func='ucb-1')

# Bayesian Optimization with Softmax using Gaussian Process
best_hyperparameters_softmax = bayesian_optimization(n_iterations=100, bound=bound, acquisition_func='softmax', temp=1.0)

print("Best Hyperparameters (UCB-1 with Gaussian Process):")
print(best_hyperparameters_ucb_1[:-1])

print("Best Hyperparameters (Softmax with Gaussian Process):")
print(best_hyperparameters_softmax)


Best Hyperparameters (UCB-1 with Gaussian Process):
[0.22213099 0.48141397]
Best Hyperparameters (Softmax with Gaussian Process):
[0.22213099 0.48141397 0.21786919]


## Discussion:

* Both exploration strategies show a convergence to a similar optimal policy, highlighting right and down actions to arrive at the goal.
* However, the Q-values for UCB-1 exploration depicts a more stable convergence, while that of softmax exploration shows a gradual convergence with some fluctuations .
* The observed policies and Q-values for the two strategies indicate a comparable performance in learning the optimal policy for the simulated grid world environment.

#### Conclusion:

* In this context, there is a notable similarity in the performance of softmax and UCB-1 exploration functions.
* Choosing between the two may depend on specific features of the environment and computational considerations, as both functions demonstrate effective learning in this context.